<a href="https://colab.research.google.com/github/fuyu-quant/langchain-examples/blob/main/ipynb/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain
* 参考記事
    * https://note.com/npaka/n/n6b7a07e492f1

In [8]:
%%capture
!pip install langchain
!pip install openai
!pip install google-search-results
!pip install wikipedia
!pip install huggingface_hub

In [3]:
# パッケージのインポート
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

import os

os.environ["LANGCHAIN_HANDLER"] = "langchain"


os.environ["OPENAI_API_KEY"] = ''
os.environ["SERPAPI_API_KEY"] = ""

In [21]:
#llm = OpenAI(temperature=0, model_name="text-davinci-002")
llm = OpenAI(temperature=0)

## ツール一覧
* python_relp
    * pythonコマンドを実行できる．
* serpapi
    * サーチエンジン，現在のイベントに関する質問に答える必要がある場合に役に立つ
* requests
    * サイトから特定のコンテンツを取得する場合に使う
    * 入力はURLで出力はそのページの全てのテキスト
* terminal
    * ターミナルでのコマンド実行
* llm-math
    * 数学の質問に答えるよう
* pal-math
    * 数学の質問に答えるよう
* pal-colored-objects
    * オブジェクトの位置と色についての推論に優れた言語モデル
* open-meteo-api
    * 気象情報を取得したい場合に使う
* news-api
    * 現在のニュース記事のトップ，ヘッドラインに関する情報を取得する
* tmdb-api
    * The Movie Databaseから情報を取得するのに使う
    

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

## エージェント一覧
* https://langchain.readthedocs.io/en/latest/modules/agents/agents.html
* どのアクションをどのような順序で行うかを決めるもの
* zero-shot-react-description
    * 
* react-docstore
    * ドキュメントベースでの回答をする
* self-ask-with-search
    * 質問に対する事実に基づく回答を検索できる
* conversational-react-description
    * 会話用に最適化されたエージェント
    * 以前の会話のやり取りを記憶できる

In [4]:
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True
)

agent.run("興味を引くようなブレインパッドの広告を10個作ってください")

In [22]:
from langchain import Wikipedia
from langchain.agents import Tool
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup
    )
]


agent = initialize_agent(
    tools, 
    llm, 
    agent="react-docstore", 
    verbose=True
)

query = "株式会社ブレインパッドについて教えてください"

agent.run(query)

TypeError: ignored

In [11]:
agent = initialize_agent(
    tools, 
    llm, 
    agent="conversational-react-description", 
    verbose=True
)

input = {}

agent.run("興味を引くようなブレインパッドの広告を10個作ってください")

ValueError: ignored

### 事実ベースの会話bot
* 会話の記憶を持たせたChat型の事実に基づく検索
* [参考](https://langchain.readthedocs.io/en/latest/modules/agents/examples/chat_conversation_agent.html)

In [26]:
from langchain.utilities import SerpAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

In [30]:
search = SerpAPIWrapper()

tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        #description="useful for when you need to answer questions about current events or the current state of the world. the input to this should be a single search term."
    ),
]

# モデルの読み込み
llm=ChatOpenAI(temperature=0)

# 記憶の追加
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    tools, 
    llm, 
    agent="chat-conversational-react-description", 
    verbose=True, 
    memory=memory
    )

In [36]:
agent.run(input="日本がWBCで勝った国はどこですか?")



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "WBC winners by year"
}
Observation: The World Baseball Classic (WBC) is an international baseball tournament, sanctioned from 2006 to 2013 by the International Baseball Federation (IBAF) and ...
Thought:{
    "action": "Final Answer",
    "action_input": "The countries that won the WBC in the years not won by Japan are:\n- 2013: Dominican Republic\n- 2017: United States"
}

> Finished chain.


'The countries that won the WBC in the years not won by Japan are:\n- 2013: Dominican Republic\n- 2017: United States'

In [33]:
agent.run(input="Which country won in the other years?")



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "WBC winners by year"
}
Observation: The World Baseball Classic (WBC) is an international baseball tournament, sanctioned from 2006 to 2013 by the International Baseball Federation (IBAF) and ...
Thought:{
    "action": "Final Answer",
    "action_input": "Here are the countries that won the WBC in the years not won by Japan:\n\n- 2013: Dominican Republic\n- 2017: United States"
}

> Finished chain.


'Here are the countries that won the WBC in the years not won by Japan:\n\n- 2013: Dominican Republic\n- 2017: United States'

In [34]:
agent.run(input="Thanks for letting me know.")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "You're welcome! Let me know if you have any other questions."
}

> Finished chain.


"You're welcome! Let me know if you have any other questions."

### 似ているサンプルの作成

In [35]:
from langchain.llms.openai import OpenAI
from langchain.example_generator import generate_example
from langchain.prompts import PromptTemplate

In [ ]:
examples = [
    {
    "question": "株式会社ブレインパッドはいつ創業されいつ創業されましたか？",
     "answer": "a"
    },
    {
    "question": "株式会社ブレインパッドはいつ創業されいつ創業されましたか？",
     "answer": "a"
    },
     {
    "question": "株式会社ブレインパッドはいつ創業されいつ創業されましたか？",
     "answer": "a"
    }
]

In [ ]:
example_template = PromptTemplate(template="Question: {question}\n{answer}", input_variables=["question", "answer"])

new_example = generate_example(examples, OpenAI(), example_template)

new_example.split('\n')

### モデルの比較

In [37]:
from langchain import LLMChain, OpenAI, Cohere, HuggingFaceHub, PromptTemplate
from langchain.model_laboratory import ModelLaboratory

In [40]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [54]:
llms = [
    OpenAI(temperature=0, model_name="text-davinci-002"), 
    #Cohere(model="command-xlarge-20221108", max_tokens=20, temperature=0), 
    HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":1}),
    # 使えないモデルもある()
    #HuggingFaceHub(repo_id="rinna/japanese-gpt-1b"),
    #HuggingFaceHub(repo_id="abeja/gpt-neox-japanese-2.7b")
]

In [55]:
model_lab = ModelLaboratory.from_llms(llms)

model_lab.compare("Tell us about LightGBM")

Input:
Tell us about LightGBM

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.0, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'best_of': 1, 'request_timeout': None, 'logit_bias': {}}


LightGBM is an open-source gradient boosting framework developed by Microsoft that uses tree-based learning algorithms. It is designed to be distributed and efficient with the use of histogram-based algorithms, which allows faster training than traditional gradient boosting techniques. LightGBM is popular for its efficiency, scalability, and accuracy. It is used in many machine learning tasks such as classification, regression, ranking, and recommendation systems. LightGBM is also used in many Kaggle competitions and has been proven to outperform other boosting algorithms.

HuggingFaceHub
Params: {'repo_id': 'google/flan-t5-xl', 'task': None, 'model_kwargs': {'temperature': 1}}
LightGBM LightGBM is a free and open source GBM player.



## ドキュメントチェーン

### ドキュメントの内容に対する質疑応答

In [ ]:
%%capture
!pip install chromadb
!pip install faiss-cpu

In [71]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
from langchain.llms import OpenAI

### Stuffing

In [82]:
llm = OpenAI(temperature=0, model_name="text-davinci-002")



with open('/content/sample.txt') as f:
    dragonball_txt = f.read()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_text(dragonball_txt)


query = "Kaggle社内Meet upとはどんなものですか?"

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)
docs = docsearch.similarity_search(query)



chain = load_qa_chain(llm, chain_type="stuff")
#chain.run(input_documents=docs, question=query)

chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' Kaggle社内Meet upは、Kaggleに参加したことがある社員や参加したことはないけれど興味がある社員、Kaggleに限らず技術コンペについて情報収集したい社員が、交流できるように企画されたイベントです。'}

### Map Reduce

In [79]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.9 MB/s eta 0:00:00


In [83]:
chain = load_qa_chain(llm, chain_type="map_reduce")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': "\n\nI don't know."}

### Refine

In [84]:
chain = load_qa_chain(llm, chain_type="refine")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

ValueError: ignored

## ソース付きの質問応答


In [85]:
docsearch = FAISS.from_texts(
    texts, 
    embeddings, 
    metadatas=[{"source": i} for i in range(len(texts))])
docs = docsearch.similarity_search(query)

In [86]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

# load_qa_with_sources_chainの準備
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

# ソース付きの質問応答の実行
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' Kaggle社内Meet upとは、Kaggleに参加したことがある社員や参加したことがないけれど興味がある社員、Kaggleに限らず技術コンペについて情報収集したい社員が、交流できるように企画されたイベントです。オフラインとオンラインの同時開催で、Kaggleの活動についてのライトニングトークや懇親会などが行われます。\nSOURCES: 0, 1, 2, 3'}

In [87]:
print(texts[0])
print(texts[1])
print(texts[2])
print(texts[3])

Kaggle社内Meet upとは？
「Kaggle」については、ご存知の方も多いと思いますが、企業や組織・団体から投稿された課題とデータをもとに、解決に適した機械学習のモデルを構築して精度を競い合うコンペ式のサービスを言います。
当社にはデータサイエンティストが約200名在籍していることもあり、データ分析の技術を研鑽するため、プライベートでKaggleに親しむKagglerが数多く在籍しています。

今回のMeet upは、「Kaggleに参加したことがある」「参加したことはないけれど興味がある」「Kaggleに限らず技術コンペについて情報収集したい」という社員同志が、交流できるように企画されたイベントです！

オフラインとオンラインの同時開催
感染症対策の考慮もしつつ、当日は本社オフィスの広い会場（セミナールーム）にて、オフラインとオンライン、どちらでも参加可能なハイブリッド形式で開催されました。
悪天候にも関わらずセミナールームには約30名が集合し、オンラインでは社員の他に新卒の内定者も加わって、総勢90名が参加する大きなイベントとなりました。
中には、懇親会を楽しみに、お客様先などから本社オフィスへ移動してくる参加者もいました。

当日のコンテンツ
前半は、Kaggle歴1年目からKaggle master（！）まで幅広い経歴の4名によるライトニングトークが行われました。

Kaggleに参加する動機、参加するコンペの選択方法、戦績、Kaggleの取り組み方、参加して得られたこと、苦しみ…など、幅広い話題が思い思いの形で発表され、笑いが起きる場面もしばしば。発表後には様々な質疑が飛び交いました。

後半は、ピザなどの軽食とドリンクなどを囲み、懇親会が開催されました。皆さん、和気あいあいと打ち解けていました。


参加者の感想
普段あまりKaggleの活動について話す機会がないため、とても充実した時間を過ごすことができた。
楽しそうに取り組んでいる様子が伝わってきて自分のモチベーションにもつながった。
まだKaggleに取り組んだことがなく、コンペの選び方などがさっぱり分からなかったので、ライトニングトークが非常に参考になった。
お客様への報告資料のようにまとめられていたところが、ブレインパッドっぽくて面白かったです。

などなど…。これらの感想のとおり、「と

## 要約

### stuff

In [90]:
from langchain.chains.summarize import load_summarize_chain

docs = [Document(page_content=t) for t in texts[:2]]

# stuffのload_qa_chainを準備
chain = load_summarize_chain(OpenAI(temperature=0), chain_type="stuff")

# 要約の実行
print(chain.run(docs))

 Kaggle社内Meet upは、Kaggleに参加したことがある社員や、参加したことがないけれど興味がある社員、Kaggleに限らず技術コンペについて情報収集したい社員を対象に、ハイブリッド形式で開催されたイベントです。前半は、Kaggle歴1年目からKaggle masterまで幅広い経歴の4名によるライトニングトークが行われ、後半は懇親会が開催されました。参加者は約90名で、参加者の感想は良好でした。


### map reduce

In [92]:
chain = load_summarize_chain(OpenAI(temperature=0), chain_type="map_reduce")
print(chain.run(docs))

 Kaggle社内Meet upは、Kaggleに参加したことがある社員や興味がある社員、技術コンペについて情報収集したい社員が交流できるイベントで、感染症対策を考慮し、オフラインとオンラインのハイブリッド形式で開催されました。90名が参加し、ライトニングトークと懇親会が行われ、参加者は和気あいあいと打ち解けていました。

CONCISE SUMMARY: Kaggle社内Meet upは、Kaggleに参加したことがある社員や


## ユーティリティチェーン
* https://note.com/npaka/n/n886960b89de1

### LLMMathChain

In [77]:
from langchain import OpenAI, LLMMathChain

# LLMの準備
llm = OpenAI(temperature=0)

# LLMMathChainの準備
llm_math = LLMMathChain(
    llm=llm, 
    verbose=True
)

# LLMMathChainの実行
print(llm_math.run("How many of the integers between 72 and 354 inclusive are divisible by 8?"))



> Entering new LLMMathChain chain...
How many of the integers between 72 and 354 inclusive are divisible by 8?
Answer: 28
> Finished chain.
Answer: 28


### PALChain

In [73]:
from langchain.chains import PALChain
from langchain import OpenAI

# LLMの準備
llm = OpenAI(
    model_name='code-davinci-002', 
    temperature=0, 
    max_tokens=512)

# PALChainの準備
pal_chain = PALChain.from_math_prompt(
    llm, 
    verbose=True
)

# PALChainの実行
# (JanはMarciaの3倍のペットを飼っています。 MarciaはCindy より2匹多くペットを飼っています。シンディが4匹のペットを飼っている場合、3 人が飼っているペットの総数は?)
question = "Jan has three times the number of pets as Marcia. Marcia has two more pets than Cindy. If Cindy has four pets, how many total pets do the three have?"
print(pal_chain.run(question))



> Entering new PALChain chain...


def solution():
    """Jan has three times the number of pets as Marcia. Marcia has two more pets than Cindy. If Cindy has four pets, how many total pets do the three have?"""
    cindy_pets = 4
    marcia_pets = cindy_pets + 2
    jan_pets = marcia_pets * 3
    total_pets = cindy_pets + marcia_pets + jan_pets
    result = total_pets
    return result

> Finished chain.
28


### SQLDatabaseChain
* データベースに対するユーザーの質問を受け取り，SQLクエリを作成しその結果を取得してユーザーに返す．

In [74]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

# SQLDatabaseの準備
db = SQLDatabase.from_uri("sqlite:///../../../notebooks/Chinook.db")

# LLMの準備
llm = OpenAI(temperature=0)

# SQLDatabaseChainの準備
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

# SQLDatabaseChainの実行
# (従業員は何人ですか？)
print(db_chain.run("How many employees are there?"))

OperationalError: ignored

### LLMBashChain

### LLMCheckerChain

### LLMRequestsChain

### OpenAIModerationChain